# Project III: A MongoDB & API EDA project

## You recently created a new company in the GAMING industry. The company will have the following scheme:

- 20 Designers
- 5 UI/UX Engineers
- 10 Frontend Developers
- 15 Data Engineers
- 5 Backend Developers
- 20 Account Managers
- 1 Maintenance guy that loves basketball
- 10 Executives
- 1 CEO/President.

As a data engineer you have asked all the employees to show their preferences on where to place the new office. Your goal is to place the new company offices in the best place for the company to grow. You have to find a place that more or less covers all the following requirements (note that it's impossible to cover all requirements, so you have to prioritize at your glance):

- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
- 30% of the company staff have at least 1 child.
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
- Account managers need to travel a lot.
- Everyone in the company is between 25 and 40, give them some place to go party.
- The CEO is vegan.
- If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
- The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.


![mapa](https://res.cloudinary.com/sblink/_/og_image.png)






### Connecting to MongoDB and importing libraries

In [1]:
import json
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
from pymongo import MongoClient
import pandas as pd
import time 


# importing the mongo.py file/library
import mongo as mongos
import api as api
import maps as maps


In [2]:
# connecting
c=mongos.connecting()


In [3]:
# calling the mongo extraction function
world_offices_data= mongos.mongo_search(c)

In [4]:
# calling the cleaning and transformation function

df_sorted=mongos.cleaning_mongo(world_offices_data)
df_sorted

# These are the 5 offices we will be working on

,company_name,total_money_raised,lat,lon,state_code,country_code,city,schools,airports,train stations,clubs,vegan places,basketball places,pet grooming
11,Carrier IQ,$42M,37.395578,-122.083329,CA,USA,Mountain View,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Digg,$45M,37.764726,-122.394523,CA,USA,San Francisco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,99designs,$35M,37.795531,-122.400598,CA,USA,San Francisco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Minted,$52.7M,37.797435,-122.403175,CA,USA,San Francisco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Netbiscuits,$27M,37.755037,-122.426442,CA,USA,San Francisco,NaN,NaN,NaN,NaN,NaN,NaN,NaN


After pushing, view your own notebook on [nbviewer](https://nbviewer.org/)

In [5]:
usa_map=maps.initial_map(world_offices_data)

/Users/miguelpalospou/Desktop/IRONHACK/Projects/Project-III-Geo/maps.py:15: UserWarning: color argument of Icon should be one of: {'lightblue', 'darkred', 'red', 'lightred', 'darkblue', 'cadetblue', 'green', 'blue', 'lightgray', 'darkgreen', 'darkpurple', 'white', 'orange', 'pink', 'gray', 'purple', 'black', 'lightgreen', 'beige'}.
  icon = Icon (


In [6]:
usa_map

### As we can see most of the startups that meet the following requirements are in the west coast:

- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.

- Developers like to be near successful tech startups that have raised at least 1 Million dollars.

### We will focus on Mountain View and San Francisco

## Let's start with the API

# Just to remember, these are the parameters we will be evaluating:
- preschool_c = "12056"
- airport_c = "19040"
- train_station_c = "19046"
- nightlife_spot_c = "10032"
- vegetarian_vegan_c = "13377"
- basketball_stadium_c = "18008"
- pet_grooming_c = "11062"


In [7]:
from getpass import getpass
from dotenv import load_dotenv
import os
import requests

In [8]:
df_sorted

,company_name,total_money_raised,lat,lon,state_code,country_code,city,schools,airports,train stations,clubs,vegan places,basketball places,pet grooming
11,Carrier IQ,$42M,37.395578,-122.083329,CA,USA,Mountain View,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Digg,$45M,37.764726,-122.394523,CA,USA,San Francisco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,99designs,$35M,37.795531,-122.400598,CA,USA,San Francisco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Minted,$52.7M,37.797435,-122.403175,CA,USA,San Francisco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Netbiscuits,$27M,37.755037,-122.426442,CA,USA,San Francisco,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Let' start with the search: <font size="4">San Francisco and Mountain View</font>

<br>
</br>
<font size="3">
    The idea here is to call a function that automatically loops through a list of categories and again through the list of potential offices (df_sorted) to create 7 dataframes, one for each category which includes all the category results for each of the 5 potential locations.he idea here is to call a function that automatically loops through a list of categories and again through the list of potential offices (df_sorted) to create 7 dataframes, one for each category which includes all the category results for each of the 5 potential locations.
</font>


<br>
</br>





In [9]:

dfs=api.categories(df_sorted)



/Users/miguelpalospou/Desktop/IRONHACK/Projects/Project-III-Geo/api.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  category_df = category_df.append(new_list, ignore_index=True)
/Users/miguelpalospou/Desktop/IRONHACK/Projects/Project-III-Geo/api.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  category_df = category_df.append(new_list, ignore_index=True)
/Users/miguelpalospou/Desktop/IRONHACK/Projects/Project-III-Geo/api.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  category_df = category_df.append(new_list, ignore_index=True)
/Users/miguelpalospou/Desktop/IRONHACK/Projects/Project-III-Geo/api.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use

## Counting number of near factors for each location
### We will now call a function that takes the 6 dataframes and counts the number of results for each location and category

In [10]:
df_sorted=api.count_category(df_sorted,dfs)


In [11]:
df_sorted

,company_name,total_money_raised,lat,lon,state_code,country_code,city,schools,airports,train stations,clubs,vegan places,basketball places,pet grooming
11,Carrier IQ,$42M,37.395578,-122.083329,CA,USA,Mountain View,2,0,0,6,3,0,0
0,Digg,$45M,37.764726,-122.394523,CA,USA,San Francisco,8,0,0,5,2,1,2
4,99designs,$35M,37.795531,-122.400598,CA,USA,San Francisco,4,0,1,10,10,0,1
7,Minted,$52.7M,37.797435,-122.403175,CA,USA,San Francisco,4,0,1,10,9,0,0
12,Netbiscuits,$27M,37.755037,-122.426442,CA,USA,San Francisco,10,0,0,9,9,1,4


In [12]:
usa_map=maps.final_map(dfs)
usa_map

/Users/miguelpalospou/Desktop/IRONHACK/Projects/Project-III-Geo/maps.py:65: UserWarning: color argument of Icon should be one of: {'lightblue', 'darkred', 'red', 'lightred', 'darkblue', 'cadetblue', 'green', 'blue', 'lightgray', 'darkgreen', 'darkpurple', 'white', 'orange', 'pink', 'gray', 'purple', 'black', 'lightgreen', 'beige'}.
  icon = Icon (
